In [ ]:
import pandas
import numpy
import matplotlib.pyplot as plt
from Bio import SeqIO
import re
import coralme
plt.rcParams['svg.fonttype'] = 'none'
pandas.set_option('display.max_colwidth', None)
%load_ext autoreload
%autoreload 2


In [ ]:
medium = pandas.read_csv("./media/M9.txt",index_col=0,header=None,comment="#")[1]

## ME-model

In [ ]:
# model = coralme.io.pickle.load_pickle_me_model("./me_model_files/KT2440-DLKcat.pkl")
me = coralme.io.pickle.load_pickle_me_model("./pputidame/MEModel-step3-pputida-Keff.pkl")

In [ ]:
for r in me.reactions.query("^EX_"):
    r.lower_bound = 0

In [ ]:
for r,b in medium.items():
    me.reactions.get_by_id(r).lower_bound = -b

## M-model

In [ ]:
import cobra
model = cobra.io.load_json_model("./me_model_files/inputs/m_model.json")

In [ ]:
for r in model.reactions.query("^EX_"):
    r.lower_bound = 0

In [ ]:
medium = pandas.read_csv("./media/M9.txt",index_col=0,header=None,comment="#")[1]
for r,b in medium.items():
    model.reactions.get_by_id(r).lower_bound = -b

## Simulations

In [ ]:
from tqdm import tqdm

In [ ]:
bounds = numpy.arange(1,12,1)
results = {"m":{b:{} for b in bounds},"me":{b:{} for b in bounds}}

In [ ]:
for b in tqdm(bounds):
    model.reactions.get_by_id("EX_glc__D_e").lower_bound = -b
    model.reactions.get_by_id("EX_glc__D_e").upper_bound = -b
    results["m"][b] = model.optimize().to_frame()["fluxes"]

In [ ]:
for b in tqdm(bounds):
    me.reactions.get_by_id("EX_glc__D_e").lower_bound = -b
    me.reactions.get_by_id("EX_glc__D_e").upper_bound = -b
    me.optimize(max_mu=1.0)
    results["me"][b] = me.solution.to_frame()["fluxes"]

In [ ]:
for i,df in results.items():
    results[i] = pandas.DataFrame.from_dict(results[i])

In [ ]:
results["m"].to_csv("./datasets/carbon-source-uptake/m-fluxes.csv")
results["me"].to_csv("./datasets/carbon-source-uptake/me-fluxes.csv")

### Compare with experimental results

In [ ]:
experimental = pandas.read_csv("./datasets/carbon-source-uptake/references.csv",index_col=0,comment="#")
experimental["In vivo"] = experimental["In vivo"].apply(lambda x:x.split("/")[0]).astype(float)
experimental = experimental.drop("Gluconate")

In [ ]:
experimental

In [ ]:
experimental["In vivo"].mean(),experimental["In vivo"].std()

In [ ]:
experimental["uptake"].mean(),experimental["uptake"].std()

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(7,5))
results["m"].loc["BIOMASS_KT2440_WT3"].plot(marker="o",ax=ax,label="m",color="#fdae61")
results["me"].loc["biomass_dilution"].plot(marker="o",ax=ax,label="me",color="#2b83ba")

for i,row in experimental.iterrows():
    plt.plot(row["uptake"],row["In vivo"],"*",color="black",markersize=10)
    plt.text(row["uptake"]+0.2,row["In vivo"]-0.01,row["Reference"])

plt.ylabel("Growth rate, 1/h")
plt.xlabel("Glucose uptake rate")
plt.xlim([0,12])
plt.legend()
plt.savefig("./figures/growth-rate-performance.svg",format="svg")

In [ ]:
from coralme.util.flux_analysis import exchange_single_model

In [ ]:
model.solution = model.optimize()

In [ ]:
for r in model.reactions:
    f = model.solution.fluxes[r.id]
    lb,ub = r.bounds
    if not f:
        continue
    if abs(f-lb) < 1e-6 or abs(f-ub) < 1e-6:
        print(r, f, lb, ub)